In [7]:
# Import dependencies
import pandas as pd
import math

In [2]:
# Read data
games = pd.read_csv('../Resources/games.csv')
pff = pd.read_csv('../Resources/pffScoutingData.csv')
players = pd.read_csv('../Resources/players.csv')
plays = pd.read_csv('../Resources/plays.csv')

week1 = pd.read_csv('../Resources/week1.csv')
week2 = pd.read_csv('../Resources/week2.csv')
week3 = pd.read_csv('../Resources/week3.csv')
week4 = pd.read_csv('../Resources/week4.csv')
week5 = pd.read_csv('../Resources/week5.csv')
week6 = pd.read_csv('../Resources/week6.csv')
week7 = pd.read_csv('../Resources/week7.csv')
week8 = pd.read_csv('../Resources/week8.csv')

tracking = week1.append([
    week2,
    week3,
    week4,
    week5,
    week6,
    week7,
    week8,
])

In [3]:
# Find tracking possession
player_tracking = tracking[tracking['team'] != 'football']
player_tracking = player_tracking.merge(plays[['gameId', 'playId', 'possessionTeam']])
player_tracking['possession'] = player_tracking['team'] == player_tracking['possessionTeam']

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,possessionTeam,possession
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,TB,True
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,TB,True
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,TB,True
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,TB,True
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,TB,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952687,2021110100,4433,53449.0,54,2021-11-02T03:20:26.500,89.0,NYG,right,49.44,48.85,1.43,3.20,0.14,270.45,2.03,None,NYG,True
7952688,2021110100,4433,53449.0,55,2021-11-02T03:20:26.600,89.0,NYG,right,49.43,48.99,1.34,2.85,0.13,273.20,349.13,None,NYG,True
7952689,2021110100,4433,53449.0,56,2021-11-02T03:20:26.700,89.0,NYG,right,49.40,49.11,1.32,2.42,0.13,275.46,338.05,None,NYG,True
7952690,2021110100,4433,53449.0,57,2021-11-02T03:20:26.800,89.0,NYG,right,49.33,49.22,1.29,2.18,0.13,277.09,326.21,None,NYG,True


In [4]:
# Reduce player_tracking to first game
game1_player_tracking = player_tracking[player_tracking['gameId'] == player_tracking['gameId'].iloc[0]]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,possessionTeam,possession
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,TB,True
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,TB,True
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,TB,True
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,TB,True
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,TB,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88611,2021090900,4772,53504.0,25,2021-09-10T03:51:00.900,97.0,DAL,right,89.07,30.29,2.92,1.59,0.30,242.80,325.15,None,TB,False
88612,2021090900,4772,53504.0,26,2021-09-10T03:51:01.000,97.0,DAL,right,88.93,30.55,2.87,1.85,0.30,235.28,330.51,None,TB,False
88613,2021090900,4772,53504.0,27,2021-09-10T03:51:01.100,97.0,DAL,right,88.80,30.83,2.92,1.84,0.31,224.02,335.31,None,TB,False
88614,2021090900,4772,53504.0,28,2021-09-10T03:51:01.200,97.0,DAL,right,88.68,31.12,2.95,1.68,0.31,210.24,338.53,None,TB,False


In [5]:
# Split opposition
offensive_player_tracking = game1_player_tracking[game1_player_tracking['possession']]
defensive_player_tracking = game1_player_tracking[~game1_player_tracking['possession']]
offensive_tracking_unfold = offensive_player_tracking.merge(defensive_player_tracking, on=['gameId', 'playId', 'frameId'], suffixes=('', '_opp'))

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,x_opp,y_opp,s_opp,a_opp,dis_opp,o_opp,dir_opp,event_opp,possessionTeam_opp,possession_opp
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,43.66,18.70,0.45,0.92,0.05,308.77,337.59,None,TB,False
1,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,44.03,32.69,0.00,0.00,0.01,265.97,215.94,None,TB,False
2,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,44.59,29.62,0.25,0.26,0.03,264.41,162.64,None,TB,False
3,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,43.49,37.55,0.19,0.09,0.07,302.07,58.43,None,TB,False
4,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,44.50,37.78,3.28,1.29,0.32,89.72,168.38,None,TB,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487383,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,86.93,34.99,1.72,2.32,0.18,62.70,316.21,None,TB,False
487384,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,96.99,46.59,4.31,4.15,0.40,321.51,336.30,None,TB,False
487385,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,102.98,40.08,5.45,1.48,0.54,352.64,50.26,None,TB,False
487386,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,87.97,31.47,2.79,1.08,0.28,349.01,301.43,None,TB,False


In [8]:
# Find distance to opposing player
ot_unfold = offensive_tracking_unfold.copy()
ot_unfold['x_diff'] = ot_unfold['x'] - ot_unfold['x_opp']
ot_unfold['x_diff'] = ot_unfold['x_diff']**2

ot_unfold['y_diff'] = ot_unfold['y'] - ot_unfold['y_opp']
ot_unfold['y_diff'] = ot_unfold['y_diff']**2

ot_unfold['diff_sum'] = ot_unfold['x_diff'] = ot_unfold['y_diff']
ot_unfold['distance_opp'] = ot_unfold['diff_sum'].apply(lambda x: math.sqrt(x))

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dis_opp,o_opp,dir_opp,event_opp,possessionTeam_opp,possession_opp,x_diff,y_diff,diff_sum,distance_opp
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.05,308.77,337.59,None,TB,False,30.4704,30.4704,30.4704,5.52
1,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.01,265.97,215.94,None,TB,False,71.7409,71.7409,71.7409,8.47
2,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.03,264.41,162.64,None,TB,False,29.1600,29.1600,29.1600,5.40
3,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.07,302.07,58.43,None,TB,False,177.6889,177.6889,177.6889,13.33
4,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.32,89.72,168.38,None,TB,False,183.8736,183.8736,183.8736,13.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487383,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.18,62.70,316.21,None,TB,False,19.8916,19.8916,19.8916,4.46
487384,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.40,321.51,336.30,None,TB,False,257.9236,257.9236,257.9236,16.06
487385,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.54,352.64,50.26,None,TB,False,91.2025,91.2025,91.2025,9.55
487386,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.28,349.01,301.43,None,TB,False,0.8836,0.8836,0.8836,0.94


In [9]:
# Extract closest match
ot_unfold.sort_values(by=['distance_opp'])
ot_unfold.drop_duplicates(subset=['gameId', 'playId', 'frameId', 'nflId'], keep='first')
ot_unfold

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dis_opp,o_opp,dir_opp,event_opp,possessionTeam_opp,possession_opp,x_diff,y_diff,diff_sum,distance_opp
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.05,308.77,337.59,None,TB,False,30.4704,30.4704,30.4704,5.52
1,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.01,265.97,215.94,None,TB,False,71.7409,71.7409,71.7409,8.47
2,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.03,264.41,162.64,None,TB,False,29.1600,29.1600,29.1600,5.40
3,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.07,302.07,58.43,None,TB,False,177.6889,177.6889,177.6889,13.33
4,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.32,89.72,168.38,None,TB,False,183.8736,183.8736,183.8736,13.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487383,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.18,62.70,316.21,None,TB,False,19.8916,19.8916,19.8916,4.46
487384,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.40,321.51,336.30,None,TB,False,257.9236,257.9236,257.9236,16.06
487385,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.54,352.64,50.26,None,TB,False,91.2025,91.2025,91.2025,9.55
487386,2021090900,4772,52421.0,29,2021-09-10T03:51:01.300,78.0,TB,right,88.39,30.53,...,0.28,349.01,301.43,None,TB,False,0.8836,0.8836,0.8836,0.94
